In [1]:
import numpy as np
import tensorflow as tf
import pickle
import keras
import preproc
from importlib import reload
import train_utils
import keras
import preproc
import train_utils
import setup_model

DATA_DIR = "data"

Using TensorFlow backend.


# Prepare training data

In [2]:
VOCAB_SIZE = 2500
data, labels = pickle.load(open(DATA_DIR + "/train_data.pyObj", "rb"))
labels = labels - 1
freq_vec = data["freq"]
stroke_vec = data["stroke"]

np.random.seed(125655)
rand_idx = np.random.permutation(range(len(labels)))
train_idx = rand_idx[:int(len(rand_idx) * 0.6)]
cv_idx = rand_idx[int(len(rand_idx) * 0.6):int(len(rand_idx) * 0.2)]
test_idx = rand_idx[int(len(rand_idx) * 0.8):]
text_mat = preproc.preproc_text_vec(data["text"], VOCAB_SIZE)

# Setup Model

In [31]:
# Model building
USE_SAVED = True
if USE_SAVED:
    model = keras.models.load_model("model.h5")
else:
    model = setup_model.setup_model(VOCAB_SIZE)

In [4]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

# Training

In [8]:
EPOCH = 10
cost_vec = []
acc_vec = []
print("Training data: %d samples" % len(train_idx))

for epoch_i in range(EPOCH):
    print(" --- Epoch {} --- ".format(epoch_i))
    counter = 0

    for sample_idx in train_idx:
        sample = text_mat[sample_idx]
        label = labels[sample_idx]
        freq = freq_vec[sample_idx]
        stroke = stroke_vec[sample_idx]
        freq = freq / np.sum(freq)
        stroke = stroke/np.sum(stroke)
        ret = model.train_on_batch(
                [np.array([sample]), np.array([freq]), np.array([stroke])], 
                keras.utils.to_categorical(label, 6))
        cost_vec.append(ret[0])
        acc_vec.append(ret[1])

        global_iter = epoch_i * len(train_idx) + counter        

        mv_cost = np.mean(cost_vec[-100:])
        mv_acc = np.mean(acc_vec[-100:])
        if (global_iter + 1) % 100 == 0:
            train_utils.write_log(tb_callback, global_iter, 
                    ["loss", "accuracy"], [mv_cost, mv_acc])

        if (global_iter + 1) % 500 == 0:                        
            print("validating model...")
            val_loss, val_acc = train_utils.test_model(
                model, test_idx, data, labels, VOCAB_SIZE)
            print("Validation: loss: {:f}, acc: {:.2f}".format(val_loss, val_acc))
            train_utils.write_log(tb_callback, global_iter, 
                    ["val_loss", "val_accuracy"], [val_loss, val_acc])    

        # if counter >= 3:
        #     break   
        print("[% 4d] cost: %s, accuracy: %.2f" % (counter, mv_cost, mv_acc))
        counter += 1

Training data: 784 samples
 --- Epoch 0 --- 
[   0] cost: 1.35231, accuracy: 1.00
[   1] cost: 1.37075, accuracy: 1.00
[   2] cost: 1.58928, accuracy: 0.67
 --- Epoch 1 --- 
[   0] cost: 1.48184, accuracy: 0.80
[   1] cost: 1.44392, accuracy: 0.83
[   2] cost: 1.51751, accuracy: 0.71
 --- Epoch 2 --- 
[   0] cost: 1.42903, accuracy: 0.78
[   1] cost: 1.37151, accuracy: 0.80
[   2] cost: 1.43588, accuracy: 0.70
 --- Epoch 3 --- 
[   0] cost: 1.20922, accuracy: 0.80
[   1] cost: 1.09501, accuracy: 0.80
[   2] cost: 1.19584, accuracy: 0.70
 --- Epoch 4 --- 
[   0] cost: 0.949924, accuracy: 0.80
[   1] cost: 0.852173, accuracy: 0.80
[   2] cost: 0.937655, accuracy: 0.70
 --- Epoch 5 --- 
[   0] cost: 0.690789, accuracy: 0.80
[   1] cost: 0.626508, accuracy: 0.80
[   2] cost: 0.772273, accuracy: 0.70
 --- Epoch 6 --- 
[   0] cost: 0.544295, accuracy: 0.80
[   1] cost: 0.506109, accuracy: 0.80
[   2] cost: 0.65539, accuracy: 0.70
 --- Epoch 7 --- 
[   0] cost: 0.458682, accuracy: 0.80
[   1]

In [14]:
np.array(freq_vec[sample_idx]).shape


(4,)

# Test Model

In [33]:
val_loss, val_acc = train_utils.test_model(
                    model, test_idx, data, labels, VOCAB_SIZE)
print("Validation: loss: {:f}, acc: {:.2f}".format(val_loss, val_acc))

Validation: loss: 2.675456, acc: 0.42


In [34]:
reload(train_utils)
ypred = train_utils.predict_model(model, test_idx, data, VOCAB_SIZE)
yans = labels[test_idx]

In [35]:
from sklearn.metrics import confusion_matrix
M6 = confusion_matrix(yans, ypred)
acc6 = np.sum(np.diag(M6)) / np.sum(M6)
print(M6)
print("Accuracy: %.4f" % acc6)


[[19  7  2  1  0  0]
 [17 16 14  3  0  0]
 [ 2  5 15 21  6  1]
 [ 0  1  5 29 16  6]
 [ 0  1  2 12 17  3]
 [ 0  0  0  8 18 15]]
Accuracy: 0.4237


In [36]:
M3 = confusion_matrix(np.int32(yans / 2), np.int32(ypred / 2))
acc3 = np.sum(np.diag(M3)) / np.sum(M3)
print(M3)
print("Accuracy: %.4f" % acc3)

[[59 20  0]
 [ 8 70 29]
 [ 1 22 53]]
Accuracy: 0.6947


# Decoding

In [50]:
import decoder
reload(decoder)
dec = decoder.Decoder("model.h5")

In [56]:
DOC_TEXT="""
觀自在菩薩。行深般若波羅蜜多時。照見五蘊皆空。度一切苦厄。舍利子。色不異空。空不異色。色即是空。空即是色。受想行識。亦復如是。舍利子。是諸法空相。不生不滅。不垢不淨。不增不減。是故空中無色。無受想行識。無眼耳鼻舌身意。無色聲香味觸法。無眼界。乃至無意識界。無無明。亦無無明盡。乃至無老死。亦無老死盡。無苦集滅道。無智亦無得。以無所得故。菩提薩埵。依般若波羅蜜多故。心無罣礙。無罣礙故。無有恐怖。遠離顛倒夢想。究竟涅槃。三世諸佛。依般若波羅蜜多故。得阿耨多羅三藐三菩提。故知般若波羅蜜多。是大神咒。是大明咒。是無上咒。是無等等咒。能除一切苦。真實不虛。故說般若波羅蜜多咒。即說咒曰。
揭諦揭諦　波羅揭諦　波羅僧揭諦　菩提薩婆訶
"""
grade = dec.predict_text(DOC_TEXT)
print(grade)

4


In [60]:
DOC_TEXT="""
既然你誠心誠意的發問了 
我們就大發慈悲的告訴你 
為了防止世界被破壞 
為了守護世界的和平 
貫徹愛與真實的邪惡 
可愛又迷人的反派角色 
武藏！ 
小次郎！ 
我們是穿梭在銀河中的火箭隊 
白洞、白色的明天正等著我們 
就是這樣喵！ 
"""
grade = dec.predict_text(DOC_TEXT)
print(grade)

4
